In [ ]:
features = ["OCCUPATION_CLASS_CD", "INSD_LAST_YEARDIF_CNT", "AGE_中", "AGE_高", "ANNUAL_PREMIUM_AMT", "APC_1ST_YEARDIF", "ANNUAL_INCOME_AMT",\
            "DIEACCIDENT_AMT", "TOOL_VISIT_1YEAR_CNT", "LAST_A_ISSUE_DT_N", "INSD_1ST_AGE_中", "EDUCATION_CD", "L1YR_A_ISSUE_CNT"]

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from xgboost import XGBClassifier, plot_importance, DMatrix
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedShuffleSplit, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap  # 原本想測試看看降維

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

train_data = pd.read_csv("train.csv", encoding="ansi", dtype={"APC_1ST_AGE":object, "RFM_R":object, "REBUY_TIMES_CNT":object})

_資料前處理_

In [ ]:
train_data["Y1"] = train_data["Y1"].map({"N":0, "Y":1})
train_data["GENDER"] = train_data["GENDER"].map({"F":0, "M":1})  # Y1轉成數字以利後面操作、性別是我覺得唯一能轉換的類別資料

predict = train_data["Y1"]
train_data = pd.get_dummies(train_data, dtype="float")  # 將所有特徵進行One-hot Encoding
train_data = train_data.astype("float")
train_data = train_data[features]  # feature選自於xgboost中plot_importance前十三名
train_data.fillna(0, inplace=True)

**比起train_test_split，它可以隨機抽取並劃分資料**

In [ ]:
sss = StratifiedShuffleSplit(test_size=0.25, train_size=0.75)
for _train, _test in sss.split(train_data, predict):
    X_train, X_test = train_data[train_data.index.isin(_train)], train_data[train_data.index.isin(_test)]
    Y_train, Y_test = predict[predict.index.isin(_train)], predict[predict.index.isin(_test)]

### __~~不管3721先丟進模型再說~~__

* 羅吉斯回歸

In [ ]:
log = LogisticRegression(solver="liblinear")
log.fit(X_train, Y_train)
roc_auc_score(Y_test, log.predict_proba(X_test)[:, 1])  # 計算AUC分數

# lb = np.percentile(log.predict_proba(X_test)[:, 1], 0.5)
# res = np.where(log.predict_proba(X_test)[:, 1]<=lb, 1, 0)
# roc_auc_score(Y_test, res)
'''
原本以為只能上傳含0跟1的結果
所以想調整閾值降低判定為1的門檻
'''

In [ ]:
cross_val_score(log, train_data, predict, cv=sss, scoring="roc_auc", n_jobs=-1)

+ [x] LinearSVC運算速度較快
- [ ] SVM算老半天算不出來

In [ ]:
svc = LinearSVC()
svc.fit(X_train, Y_train)
# roc_auc_score(Y_test, log.decision_function(X_test)[:, 1])

svc.decision_function(X_test)  # decision_function可以計算每筆資料與boundary的距離[但我不會用]

> 決策樹的上傳結果比訓練成果來的差  
> 隨機森林經測試效果又更差(我的測試啦)
>> 但這兩個模型都能畫出美美的圖！！！

In [ ]:
dtc = DecisionTreeClassifier(max_depth=6, min_samples_leaf=2)
dtc.fit(X_train, Y_train)
roc_auc_score(Y_test, dtc.predict_proba(X_test)[:, 1])

In [ ]:
cross_val_score(dtc, train_data, predict, cv=sss, scoring="roc_auc", n_jobs=-1)

### sklearn內建的神經網路

In [ ]:
mlp = MLPClassifier()
mlp.fit(X_train, Y_train)
mlp.score(X_test, Y_test)  # 計算平均準確率

In [ ]:
cross_val_score(mlp, train_data, predict, cv=sss, scoring="roc_auc", n_jobs=-1)

### 上傳資料

In [ ]:
test_data = pd.read_csv("test.csv", encoding="ansi", dtype={"APC_1ST_AGE":object, "RFM_R":object, "REBUY_TIMES_CNT":object})
test_data["GENDER"] = test_data["GENDER"].map({"F":0, "M":1})
test_data = test_data.set_index("CUS_ID")
test_data = pd.get_dummies(test_data, dtype="float")
''''''
test_data = test_data[features]
test_data.fillna(0, inplace=True)
''''''
test_data = test_data.astype("float")

In [ ]:
mlp_pred = pd.Series(mlp.predict_proba(test_data)[:, 1], name="Ypred")
upload = pd.concat([pd.Series(test_data.index), mlp_pred], axis=1).set_index("CUS_ID")
upload.to_csv("upload_0926_1.csv")